In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import OneHotEncoder
import math
torch.manual_seed(0)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
torch.cuda.is_available() 

True

# Dataloader

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Custom Dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data.float()

    def __getitem__(self, index):
        return self.data[index].float()

    def __len__(self):
        return len(self.data)

In [5]:
def train_epoch(train_loader, model, optimizer, loss_function):
    losses = []
    # get a batch of training data from the train_loader (DataLoader obj)
    for i, data in enumerate(train_loader):
        x = data[:,:-1].to(device)
        t = data[:,-1].unsqueeze(1).to(device)
        # make predictions for this batch
        y = model(x)
        # Compute the loss
        loss = loss_function(y, t)
        # zero out the gradients so that it will not accumulate through each iteration
        optimizer.zero_grad()
        # Compute the gradents with the backward call (backprop)
        loss.backward()
        # Update weight using gradient descent 
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_epoch(valid_loader, model, loss_function):
    rmses = []
    with torch.no_grad(): 
        for i, data in enumerate(valid_loader):
            x = data[:,:-1].to(device)
            t = data[:,-1].unsqueeze(1).to(device)
            # Compute prediction
            y = model(x)
            
            rmse = math.sqrt(loss_function(y,t))
            rmses.append(rmse)
            
    return np.mean(rmses)

# Define Model Architectures

In [6]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.selu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(104, 64)
        # self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32,16)
        self.fc5 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.selu(self.fc1(x))
        x = self.dropout(x)
        # x = self.dropout(self.relu(self.fc2(x)))
        x = self.selu(self.fc3(x))
        x = self.dropout(x)
        x = self.selu(self.fc4(x))
        x = self.dropout(x)
        x = self.fc5(x)
        return x

# Create an instance of the neural network
model = Net()
model = model.to(device)
criterion = nn.MSELoss()

# Load in Data

In [7]:
df_np = np.load('data/processed_train_set.npy')
df_np.shape

(1710670, 105)

In [8]:
# load in np array and DRAW SAMPLE
num_samples = int(df_np.shape[0]*1)

num_rows = df_np.shape[0]
sampled_indices = np.random.choice(num_rows, size=num_samples, replace=False)

In [9]:
# convert np to tensor
df = torch.tensor(df_np[sampled_indices])

In [10]:
df.shape

torch.Size([1710670, 105])

In [11]:
# Create a sample dataset
dataset = MyDataset(df)

# Define the sizes of train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset)-train_size

In [12]:
# Split the dataset
batch_size = 256
train_data, val_data = random_split(dataset, [train_size, val_size])

# Create DataLoader objects for train, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_data, batch_size=batch_size)
# test_loader = DataLoader(test_data, batch_size=32)

# Train Model

In [13]:
def train_model(model, model_name, epochs, lr, batch_size):
    # Define the optimizer
    train_rmses = []
    valid_rmses = []
    optimizer = torch.optim.Adam(model.parameters(), lr)
#     optimizer = torch.optim.Adam(model.parameters())
    train_accs, valid_accs = [], []
    for epoch in range(epochs):

        model.train() # gradient tracking is on

        train_loss = train_epoch(train_loader, model, optimizer, criterion)

        model.eval() # we don't need gradients on to do reporting

        train_rmse = eval_epoch(train_loader, model, criterion)
        valid_rmse = eval_epoch(valid_loader, model, criterion)
        
#         train_rmses.append(train_rmse)
#         valid_rmses.append(valid_rmse)

        print(f"Epoch: {epoch+1}, Train Loss: {train_loss:>0.4f}, Train mse: {train_rmse:>0.2f},\
              Validation Accuracy: {valid_rmse:>0.2f} \n")
    torch.save(model.state_dict(), './{}'.format(model_name))


In [14]:
%%time
train_model(model = model, model_name = 'main_model', \
epochs = 20, lr=0.001, batch_size = batch_size)

Epoch: 1, Train Loss: 526412.2720, Train mse: 625.81,              Validation Accuracy: 624.48 

Epoch: 2, Train Loss: 511947.3422, Train mse: 623.75,              Validation Accuracy: 622.15 

Epoch: 3, Train Loss: 508520.5465, Train mse: 623.22,              Validation Accuracy: 622.86 

Epoch: 4, Train Loss: 503477.8185, Train mse: 620.97,              Validation Accuracy: 620.28 

Epoch: 5, Train Loss: 499021.0595, Train mse: 619.80,              Validation Accuracy: 618.66 

Epoch: 6, Train Loss: 496479.9544, Train mse: 619.91,              Validation Accuracy: 619.21 

Epoch: 7, Train Loss: 493808.2192, Train mse: 618.93,              Validation Accuracy: 617.92 

Epoch: 8, Train Loss: 491088.7089, Train mse: 619.56,              Validation Accuracy: 619.30 

Epoch: 9, Train Loss: 489625.2760, Train mse: 618.31,              Validation Accuracy: 617.87 

Epoch: 10, Train Loss: 487941.9507, Train mse: 619.15,              Validation Accuracy: 617.90 

Epoch: 11, Train Loss: 487187

In [15]:
# saved_model = Net()
# saved_model.load_state_dict(torch.load('./cse151B_project'))
# saved_model

# Test Model

In [15]:
df_np_test = np.load('data/processed_test_set.npy')
df_np_test.shape

(320, 104)

In [16]:
test_tensor = torch.tensor(df_np_test).to(device)
test_tensor.shape

torch.Size([320, 104])

In [17]:
model = model.float()
model

Net(
  (selu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=104, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=1, bias=True)
)

In [18]:
pred = []
with torch.no_grad(): 
    for i in range(test_tensor.shape[0]):
        x = test_tensor[i].float().to(device)
        y = model(x)
        pred.append(y.cpu().numpy()[0])         

In [19]:
pred

[754.574,
 676.2228,
 754.574,
 669.6948,
 696.5055,
 695.2477,
 754.574,
 695.2477,
 739.2016,
 754.574,
 613.3611,
 786.6452,
 786.6452,
 786.6452,
 786.6452,
 786.6452,
 786.6452,
 786.6452,
 754.574,
 786.6452,
 754.574,
 695.2477,
 769.6172,
 754.574,
 630.18616,
 638.19617,
 754.574,
 754.574,
 639.3683,
 754.574,
 754.574,
 676.2228,
 786.6452,
 786.6452,
 786.6452,
 786.6452,
 792.55304,
 608.1158,
 754.574,
 657.70703,
 644.154,
 786.6452,
 786.6452,
 739.2016,
 638.19617,
 634.75714,
 786.6452,
 786.6452,
 559.09174,
 613.3611,
 604.95844,
 634.0281,
 608.1158,
 754.574,
 754.574,
 620.349,
 739.2016,
 626.8517,
 786.6452,
 695.2477,
 754.574,
 786.6452,
 786.6452,
 792.55304,
 739.2016,
 739.2016,
 754.574,
 695.2477,
 786.6452,
 604.95844,
 781.88385,
 786.6452,
 754.574,
 786.6452,
 606.4421,
 925.8481,
 615.4166,
 925.8481,
 925.8481,
 925.8481,
 692.501,
 606.4421,
 692.501,
 692.501,
 637.2751,
 692.501,
 925.8481,
 925.8481,
 606.4421,
 692.501,
 792.74365,
 925.8481,


In [20]:
test_df = pd.read_csv('data/test_public.csv')
test_data = {'TRIP_ID': test_df['TRIP_ID'],
    'TRAVEL_TIME': pred}
output = pd.DataFrame(data = test_data)
output

,TRIP_ID,TRAVEL_TIME
0,T1,754.573975
1,T2,676.222778
2,T3,754.573975
3,T4,669.694824
4,T5,696.505493
...,...,...
315,T323,624.821655
316,T324,638.426025
317,T325,711.129211
318,T326,624.821655


In [57]:
output.to_csv('data/nathan_test_june7.csv', index=False)


In [56]:
torch.save(model.state_dict(), "data/model_weights.pt")